<a href="https://colab.research.google.com/github/mohamedasharf311/Handwritten-Digit-Recognition-Model/blob/main/Customer_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    for name in dirs:
        print("Folder:", os.path.join(root, name))

Folder: /content/drive/MyDrive/Colab Notebooks
Folder: /content/drive/MyDrive/COVID-19_Radiography_Dataset
Folder: /content/drive/MyDrive/Viral Pneumonia
Folder: /content/drive/MyDrive/Normal
Folder: /content/drive/MyDrive/Lung_Opacity
Folder: /content/drive/MyDrive/COVID
Folder: /content/drive/MyDrive/Xray_Data_Organized
Folder: /content/drive/MyDrive/Viral Pneumonia/masks
Folder: /content/drive/MyDrive/Viral Pneumonia/images
Folder: /content/drive/MyDrive/Normal/masks
Folder: /content/drive/MyDrive/Normal/images
Folder: /content/drive/MyDrive/Lung_Opacity/masks
Folder: /content/drive/MyDrive/Lung_Opacity/images
Folder: /content/drive/MyDrive/COVID/masks
Folder: /content/drive/MyDrive/COVID/images
Folder: /content/drive/MyDrive/Xray_Data_Organized/COVID
Folder: /content/drive/MyDrive/Xray_Data_Organized/Normal
Folder: /content/drive/MyDrive/Xray_Data_Organized/Lung_Opacity
Folder: /content/drive/MyDrive/Xray_Data_Organized/Viral_Pneumonia


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import files
from tensorflow.keras.preprocessing import image

# إعداد البيانات
dataset_path = "/content/drive/MyDrive/Xray_Data_Organized"
batch_size = 32
img_size = (224, 224)

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

# بناء النموذج
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = True

# Fine-tune: نجمد الطبقات الأولى ونفتح آخر 50 فقط
for layer in base_model.layers[:-50]:
    layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = Model(inputs, outputs)

# تدريب النموذج
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# حفظ النموذج
model.save("xray_mobilenetv2_model.keras")

# التنبؤ على صورة من الجهاز
uploaded = files.upload()
for fn in uploaded.keys():
    img_path = fn

img = image.load_img(img_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

pred = model.predict(img_array)
predicted_class = class_names[np.argmax(pred)]
confidence = 100 * np.max(pred)

plt.imshow(img)
plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
plt.axis("off")
plt.show()

# Confusion Matrix
y_true = []
y_pred = []
for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# تصدير إلى TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('xray_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("\nTFLite model saved as 'xray_model.tflite'")

Found 21181 files belonging to 4 classes.
Using 16945 files for training.
Found 21181 files belonging to 4 classes.
Using 4236 files for validation.
Classes: ['COVID', 'Lung_opacity', 'Normal ', 'Viral Pneumonia']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
116/530 ━━━━━━━━━━━━━━━━━━━━ 33:06 5s/step - accuracy: 0.6220 - loss: 0.8861